GNM + RMSD vs xtal structure

In [2]:
import os
#os.getcwd()
os.chdir(r'/home/jacobnorth/Documents/experiments/JLN_1_18d/to_mdanaly') # absolute path includes r prefix

In [3]:
#import 
import MDAnalysis as mda
from MDAnalysis.analysis import gnm
from MDAnalysis.analysis import rms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

In [4]:
# variables to permute
sim_modl=["acetone", "cyp", "fab"]     # names of pdb-type models in PWD/modl verified pre-simulation
#sim_modl=(acetone)     # names of pdb-type models in PWD/modl verified pre-simulation
sim_reps=["1"]     # number of replicates to run each simulation for
sim_tmps=["298", "310"]
#sim_tmps=(298)
#sim_solv=(SPC AON)     # names of solvents in PWD/solv verified pre-simulation
sim_solv=["SPC"]     # names of solvents in PWD/solv verified pre-simulation
sim_ffld=["G54a7"]     # names of force fields in PWD/ffld (?) verified pre-simulation
sim_stps=["5000000"]     # length of time
#sim_ligs=(None)     # names of ligands (can be array of arrays!) in PWD/ligs verified pre-simulation

u_labls=list()

# generate figure label names
for i in sim_modl:
    for j in sim_reps:
        for k in sim_tmps:
            for l in sim_solv:
                for m in sim_ffld:
                    for n in sim_stps:
                        u_labls.append(str(i)+"_"+str(j)+"_"+str(k)+"_"+str(l)+"_"+str(m)+"_"+str(n))
#print(u_labls)         # show the labels generated

In [31]:
# prepare universe(s)
unis = list()      # create list of universes
refs = list()    # create list of reference universes
r_labls = ['Acetone','CypA','FAB']       # reference xtal structure labels
#labells = ['298K','298K','298K','298K','490K','490K','490K','490K']
#fig_labl = ['Ace_298_SPC_Ace','Ace_298_SPC_Cyp','Ace_298_SPC_FAB','Ace_490_SPC_Ace','Ace_490_SPC_Cyp','Ace_490_SPC_FAB']

#'convention=STRUCT_T_SOLV_RMSDvs'

fig_labls = list()

# generate full combo labels
for i in u_labls:
    for j in r_labls:
        fig_labls.append(str(i)+"_"+str(j))

#print(fig_labls)
#print(len(fig_labls))

In [32]:
# add target universes
u_filenames = list()

# ADD AUTOMATICALLY 
for filename in os.listdir(path="."):
    if filename.endswith(".xtc"):       # for all .xtc files in the folder "to_mdanaly",
        # append a universe containing the same gro and xtc files
        curr_fn = filename.replace(".xtc","")
        u_filenames.append(curr_fn)    # create a file prefix common to the topology and trajectory files
        unis.append(mda.Universe(curr_fn+".gro", curr_fn+".xtc"))

# ADD BY HAND
#unis.append(mda.Universe("md_0_1_298.gro","md_0_1_298.xtc"))
#unis.append(mda.Universe("md_0_1_490.gro","md_0_1_490.xtc"))
#unis.append()

# for all filenames ending with .xtc (these are trajectories),
# extract the name of the prefix, assign it to a labels list.

In [7]:
#print(filenames)

['acetone_1_298_SPC_G54a7_5000000', 'fab_1_298_SPC_G54a7_5000000', 'acetone_1_310_SPC_G54a7_5000000', 'fab_1_310_SPC_G54a7_5000000', 'cyp_1_310_SPC_G54a7_5000000', 'cyp_1_298_SPC_G54a7_5000000']


In [33]:
# reference universes for xtal RMSD comparison

r_filenames = list()
pathname = "gro_rmsdstructs/noh/"

# for all .gro files in the folder, "to_mdanaly/gro_rmsdstructs",
# append the universe containing those .gro files

for filename in os.listdir(pathname):
    if filename.endswith(".gro"):       # for all .xtc files in the folder "to_mdanaly",
        # append a universe containing the same gro and xtc files
        curr_fn = filename.replace(".gro","")
        r_filenames.append(curr_fn)    # create a file prefix common to the topology and trajectory files
        refs.append(mda.Universe(pathname+curr_fn+".gro"))


In [34]:
#refs.append(mda.Universe("csa_solv_ions.gro"))     # define the reference universe
#refs.append(mda.Universe("gro_rmsdstructs/csa_acetone_noh.gro"))    # def ref unis
#refs.append(mda.Universe("gro_rmsdstructs/csa_cyp_noh.gro"))    # def ref unis
#refs.append(mda.Universe("gro_rmsdstructs/csa_fab_noh.gro"))    # def ref unis

for r in refs:
    r.trajectory[-1] # set to last frame

In [46]:
# prepare GNM
nma = list()           
eigenvalues = list()    
#time = list()
nma_cb = list()
ev_cb = list()

# calpha atoms (backbone)
for i in list(range(len(unis))):    # for the number of plots to be made on the same graph,
    nma.append(gnm.GNMAnalysis(unis[i], select='name CA', cutoff=7.0))    # create,
    nma[i].run()            #  run the analyses
    eigenvalues.append([res[1] for res in nma[i].results])      # calculate eigenvalues
    #time.append([res[0] for res in nma[i].results])             # calculate times

# sidechain atoms
for i in list(range(len(unis))):    # for the number of plots to be made on the same graph,
    nma_cb.append(gnm.GNMAnalysis(unis[i], select='name CB', cutoff=7.0))    # create,
    nma_cb[i].run()            #  run the analyses
    ev_cb.append([res[1] for res in nma_cb[i].results])      # calculate eigenvalues
    #time.append([res[0] for res in nma[i].results])             # calculate times

In [40]:
xlim = 1001
time = list(range(xlim))       # fix time differences (scale by 10)

In [10]:
#sel = 'not backbone and not type H and resnum 2:' 
#u = mda.Universe("md_0_1_490.gro","md_0_1_490.xtc")
sel = 'resnum 1 or resnum 2 or resnum 3 and name CB'
print('Target uni:\t'+str(len(u.select_atoms(sel).positions)))
for r in refs:
    print('Reference uni:\t'+str(len(r.select_atoms(sel).positions)))

Target uni:	3
Reference uni:	3
Reference uni:	3
Reference uni:	3


In [37]:
# prepare RMSD vs xtal structure
bb_rmss = list()
bb_rmsds = list()
bb_data = list()
sc_rmss = list()
sc_rmsds = list()
sc_data = list()

# backbone atoms
for u in unis:    # for all universes,
    for r in refs:  # for all reference structures,
        bb_rmss.append(rms.rmsd(u.select_atoms('backbone').positions, r.select_atoms('backbone').positions, center=True, superposition=True))  # append the rms analysis to the given position in rmss
        bb_rmsds.append(rms.RMSD(u, r, select='backbone', ref_frame=0))   # run RMSD analyses
for analysis in bb_rmsds:
    analysis.run()
    bb_data.append(pd.DataFrame(analysis.rmsd, columns=['Frame', 'Time (ps)', 'Backbone']))

# sidechain atoms
#sel = str(protein)      # normal protein
#sel = str()            # using selection algebra
#sel = 'resnum 1:10 and not type H'            # resnumbers
sel = 'backbone'

for u in unis:    # for all universes,
    for r in refs:  # for all reference structures,
        sc_rmss.append(rms.rmsd(u.select_atoms(sel).positions, r.select_atoms(sel).positions, center=True, superposition=True))  # append the rms analysis to the given position in rmss
        sc_rmsds.append(rms.RMSD(u, r, select=sel, ref_frame=0))   # run RMSD analyses
for analysis in sc_rmsds:
    analysis.run()
    sc_data.append(pd.DataFrame(analysis.rmsd, columns=['Frame', 'Time (ps)', 'Residue']))

In [38]:
#bb_evs = [eigenvalues[0],eigenvalues[0],eigenvalues[0],eigenvalues[1],eigenvalues[1],eigenvalues[1]]
#sc_evs = [eigenvalues[0],eigenvalues[0],eigenvalues[0],eigenvalues[1],eigenvalues[1],eigenvalues[1]]

bb_evs = list()

# create backbone eigenvalue sets. Should be replicated for each reference structure
for i in list(range(len(unis))):
    for j in list(range(len(refs))):
        bb_evs.append(eigenvalues[i])       # append the eigenvalue j times
#print(bb_evs)

34913139015, 1.7603754093552382, 1.9676459546293714, 1.8912334913139015, 1.511141722415819, 1.327188392774108, 1.871521794672373, 1.687558714001799, 1.9676459546293714, 1.6189996242629128, 1.9676459546293714, 1.790548800195784, 1.699204279564456, 1.738694320751109, 1.871521794672373, 1.7994588496492139, 1.871521794672373, 1.482048445903755, 1.871521794672373, 1.6713693621665506, 1.9676459546293714, 1.738694320751109, 1.9676459546293714, 1.8324455158691233, 1.327188392774108, 1.6339933351755604, 1.7564966182440835, 1.738694320751109, 1.3374794200982156, 1.871521794672373, 1.5751750597755076, 1.6791239982884365, 1.6200289910214298, 1.6200289910214298, 1.8121552076822753, 1.4311529763914217, 1.738694320751109, 1.3374794200982156, 1.699204279564456, 1.6189996242629128, 1.871521794672373, 1.9676459546293714, 1.6189996242629128, 1.8324455158691233, 1.7994588496492139, 1.8912334913139015, 1.6588935045084674, 1.9676459546293714, 1.5606022395789723, 1.9676459546293714, 1.8912334913139015, 1.832

In [45]:
# prepare individual plots

for i in list(range(len(bb_rmss))):
    ax = bb_data[i].plot(x='Frame', y=['Backbone'], kind='line', figsize=(100,5), xlim=(-1,xlim))
    #ax = sc_data[i].plot(x='Frame', y=['Backbone'], kind='line', figsize=(100,5), xlim=(-1,xlim))
    ax.set_ylabel('RMSD ($\AA$)')
    ax.set_xlabel('Time (ps)')
    #ax.xticks((1000*np.array(list(range(1,51)))))
    #ax.figure.savefig("rmsd_vs_acetone.pdf", bbox_inches="tight")
    ax2=ax.twinx()
    ax2.plot(time, bb_evs[i], label=fig_labls, color='red')    # fixed for now!!! how do I make this oscillate properly??? make a 2D array of times/eigenvals/labels?
    #ax.figure.savefig("plt"+str(i)+".pdf", bbox_inches="tight")

ValueError: x and y must have same first dimension, but have shapes (1001,) and (8,)

In [ ]:
# TODO: 
# * compare rmsds of starting structures (xtal)
# * calculate RMSD of sidechains also?

In [ ]:
dlist = list()

for i in bb_data:
    dlist.append(i.values.tolist())

In [ ]:
#NECESSARY: parse the dataframes for precisely the data you need! test it out if you aren't sure...

#bb_data[1].plot(x='Time (ps)',)

# select the time and backbone w/in the dataframe,
df_times = bb_data[0]['Time (ps)']
df_bbrmsd = bb_data[0]['Backbone']

# then convert to lists!
t_list = df_times.values.tolist()
bbrmsd = df_bbrmsd.values.tolist()

In [42]:
#pre-process/procure info from df
bb_rmsds = list()
for i in list(range(len(bb_rmss))):
    bb_rmsds.append(bb_data[i]['Backbone'])

# prepare large subplot
fig, axs = plt.subplots(6,1, sharex=True, sharey=True)
for i in list(range(len(axs))):
    #axs[i].plot(bb_data[i])
    #axs[i].plot(x='Frame', y=['Backbone'], kind='line')
    axs[i].plot(t_list, bb_rmsds[i])

    axs2=axs[i].twinx()
    axs2.plot(t_list, bb_evs[i], label=fig_labl[i], color='red')
    #axs.xlim(-1,5001)
    axs[i].set_xlim([-1,xlim])      # set x limit for figure    
    #axs[i].legend(loc=3)
    axs[i].set_ylabel('RMSD ($\AA$)')
    axs[i].set_xlabel('Time (ps)')
    axs[i].set_title(fig_labl[i], loc='left')       # label with the specified labels (automate later!); move the title to the LHS of the page
    #axs[i].xticks(list(range(0,1000,50000)))

fig.set_size_inches(100,15)
fig.suptitle('Coincidence of RMSD relative to xtal structure (red) with conformational eigenvalue (blue) versus time (ps). convention=STRUCT_T_SOLV_RMSDvs')
fig.savefig("subplots.pdf", bbox_inches="tight")

NameError: name 't_list' is not defined